# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../WeatherPy/cities_data_file.csv"
cities_weather_df = pd.read_csv(file_path)
cities_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,lebu,12.0,CL,1.620542e+09,91.0,-37.6167,-73.6500,52.88,12.73
1,1,puerto ayora,59.0,EC,1.620542e+09,93.0,-0.7393,-90.3518,75.00,5.01
2,2,faanui,100.0,PF,1.620542e+09,74.0,-16.4833,-151.7500,80.60,8.48
3,3,upernavik,100.0,GL,1.620542e+09,96.0,72.7868,-56.1549,25.05,3.38
4,4,atuona,11.0,PF,1.620542e+09,71.0,-9.8000,-139.0333,80.28,17.63
...,...,...,...,...,...,...,...,...,...,...
565,565,solovetskiy,98.0,CN,1.620543e+09,52.0,23.2281,110.8261,91.45,9.01
566,566,windhoek,100.0,RU,1.620543e+09,97.0,61.3314,40.0858,37.90,7.54
567,567,caravelas,100.0,CA,1.620543e+09,80.0,49.8834,-106.5512,35.60,6.91
568,568,mayumba,42.0,RU,1.620543e+09,47.0,60.9058,61.5186,67.48,4.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(g_key)

In [ ]:
locations = cities_weather_df[["Latitude", "Longitude"]]
humidity = cities_weather_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10, 
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
